![5 Days Open Course on Practical Blockchain](http://localhost/5_days_bc.png)

# Building A Minimilistic Blockchain


In [1]:
# Let's define our Block first

import hashlib as hasher

class Block:
  def __init__(self, index, timestamp, data, previous_hash):
    self.index = index
    self.timestamp = timestamp
    self.data = data
    self.previous_hash = previous_hash
    self.hash = self.hash_block()
  
  def hash_block(self):
    sha = hasher.sha256()
    sha.update(str(self.index) + 
               str(self.timestamp) + 
               str(self.data) + 
               str(self.previous_hash))
    return sha.hexdigest()

In [2]:
import datetime as date

# creating a simple genesis block

def create_genesis_block():
    return Block(0, date.datetime.now(), "Genesis Block", "0")

In [3]:
# generating new blocks for the chain

def next_block(last_block):
    this_index = last_block.index + 1
    this_timestamp = date.datetime.now()
    this_data = "Hey! I'm block " + str(this_index)
    this_hash = last_block.hash
    return Block(this_index, this_timestamp, this_data, this_hash)

In [5]:
# Create the blockchain and add the genesis block
blockchain = [create_genesis_block()]
previous_block = blockchain[0]

# How many blocks should we add to the chain
# after the genesis block
num_of_blocks_to_add = 20

# Add blocks to the chain
for i in range(0, num_of_blocks_to_add):
    block_to_add = next_block(previous_block)
    blockchain.append(block_to_add)
    previous_block = block_to_add
    
    print "Block #{} has been added to the blockchain!".format(block_to_add.index)
    print "Hash: {}\n".format(block_to_add.hash) 

Block #1 has been added to the blockchain!
Hash: 0beca1518ed35955d9975e111ca51763cb96a51ac58144c0da2c1a368ded9dce

Block #2 has been added to the blockchain!
Hash: 2f912f8f13825d4b2cb4b4b6862539d81ea62af2d6832b067bd6ef85e49816dd

Block #3 has been added to the blockchain!
Hash: 14996ae360de4443ed316c42bb68fbc604609f3b4a47c9649d4c66a2f96de1bc

Block #4 has been added to the blockchain!
Hash: bdb9b9de17935dcdbe27bb9d6d247e4a7c5459fb379bca0859b89da33b1a4d84

Block #5 has been added to the blockchain!
Hash: 2a9dae7af1c753fb8ba8df63f08b1dd911c39c6c6b58b34fa9b09e3254226621

Block #6 has been added to the blockchain!
Hash: 35c7dc555206d93d6c01f3c148903df2eb9b835af5c26490f9eb7bd525d2038d

Block #7 has been added to the blockchain!
Hash: 908eee044706b296af756b5f81876c56c582c6dcab88418c3ce4941b10dc7ea5

Block #8 has been added to the blockchain!
Hash: 954323fa4524f12870fd544cbaf378847c4efb976a8852fa054a8fb076e04095

Block #9 has been added to the blockchain!
Hash: 73f7855c86c4953548343e23af28e2d

In [27]:
from flask import Flask
from flask import request
import json

node = Flask(__name__)

# Store the transactions that
# this node has in a list
this_nodes_transactions = []

@node.route('/txion', methods=['POST'])
def transaction():
  if request.method == 'POST':
    # On each new POST request,
    # we extract the transaction data
    new_txion = request.get_json()
    # Then we add the transaction to our list
    this_nodes_transactions.append(new_txion)
    # Because the transaction was successfully
    # submitted, we log it to our console
    print "New transaction"
    print "FROM: {}".format(new_txion['from'])
    print "TO: {}".format(new_txion['to'])
    print "AMOUNT: {}\n".format(new_txion['amount'])
    # Then we let the client know it worked out
    return "Transaction submission successful\n"

miner_address = "q3nf394hjg-random-miner-address-34nf3i4nflkn3oi"

def proof_of_work(last_proof):
  # Create a variable that we will use to find
  # our next proof of work
  incrementor = last_proof + 1
  # Keep incrementing the incrementor until
  # it's equal to a number divisible by 9
  # and the proof of work of the previous
  # block in the chain
  while not (incrementor % 9 == 0 and incrementor % last_proof == 0):
    incrementor += 1
  # Once that number is found,
  # we can return it as a proof
  # of our work
  return incrementor

blockchain = []

def Block(index, timestamp, data, prev_hash):
    sha = hasher.sha256()
    sha.update(str(index) + 
               str(timestamp) + 
               str(data) + 
               str(prev_hash))
    return {'index': index, 'timestamp': str(timestamp), 'data': data, 'prev_hash': prev_hash, 'hash': sha.hexdigest()}

@node.route('/genesis_block', methods = ['GET'])
def genesis_block():
    if(len(blockchain) == 0):
        block = Block(0,date.datetime.now(),{'transactions': [], 'proof-of-work': 1},'0')
        #block = {'index': 0, 'data':{'transactions': [], 'proof-of-work': 1}, 'hash': '', 'timestamp': date.datetime.now()}
        blockchain.append(block)
        return 'Genesis Block Created'
    else:
        return 'Genesis Exists'

@node.route('/mine', methods = ['GET'])
def mine():
  # Get the last proof of work
  last_block = blockchain[len(blockchain) - 1]
  print last_block
  last_proof = last_block['data']['proof-of-work']
  # Find the proof of work for
  # the current block being mined
  # Note: The program will hang here until a new
  #       proof of work is found

  proof = proof_of_work(last_proof)
  # Once we find a valid proof of work,
  # we know we can mine a block so 
  # we reward the miner by adding a transaction
  this_nodes_transactions.append(
    { "from": "network", "to": miner_address, "amount": 1 }
  )
  # Now we can gather the data needed
  # to create the new block
  new_block_data = {
    "proof-of-work": proof,
    "transactions": list(this_nodes_transactions)
  }
  new_block_index = last_block['index'] + 1
  new_block_timestamp = this_timestamp = date.datetime.now()
  last_block_hash = last_block['hash']
  # Empty transaction list
  this_nodes_transactions[:] = []
  # Now create the
  # new block!
  mined_block = Block(
    new_block_index,
    new_block_timestamp,
    new_block_data,
    last_block_hash
  )
  blockchain.append(mined_block)
  # Let the client know we mined a block
  return json.dumps({
      "index": new_block_index,
      "timestamp": str(new_block_timestamp),
      "data": new_block_data,
      "hash": last_block_hash
}) + "\n"

@node.route('/blocks', methods=['GET'])
def get_blocks():
  chain_to_send = blockchain
  # Convert our blocks into dictionaries
  # so we can send them as json objects later
  #for block in chain_to_send:
  #  block_index = str(block.index)
  #  block_timestamp = str(block.timestamp)
  #  block_data = str(block.data)
  #  block_hash = block.hash
  #  block = {
  #    "index": block_index,
  #    "timestamp": block_timestamp,
  #    "data": block_data,
  #    "hash": block_hash
  #  }
  # Send our chain to whomever requested it
  #chain_to_send = json.dumps(chain_to_send)
  return json.dumps(chain_to_send)

def find_new_chains():
  # Get the blockchains of every
  # other node
  other_chains = []
  for node_url in peer_nodes:
    # Get their chains using a GET request
    block = requests.get(node_url + "/blocks").content
    # Convert the JSON object to a Python dictionary
    block = json.loads(block)
    # Add it to our list
    other_chains.append(block)
  return other_chains

def consensus():
    # Get the blocks from other nodes
    other_chains = find_new_chains()
    # If our chain isn't longest,
    # then we store the longest chain
    longest_chain = blockchain
    for chain in other_chains:
        if len(longest_chain) < len(chain):
            longest_chain = chain
    # If the longest chain wasn't ours,
    # then we set our chain to the longest
    blockchain = longest_chain

node.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
